In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# Read data from file
with open('data.txt', 'r') as f:
    data = f.readlines()

# Define player data
players = []
for i in range(0, len(data), 3):
    player = {
        'defense_ranks': np.array([int(j) for j in data[i].split(',')], dtype=np.float32),
        'fantasy_scores': np.array([float(j) for j in data[i+1].split(',')], dtype=np.float32),
        'skill_score': np.array([float(j) for j in data[i+2].split(',')], dtype=np.float32),
    }
    players.append(player)

# Process inputs and targets for each player
X_train, X_test, Y_train, Y_test = [], [], [], []
 
for player in players:
    player_inputs = torch.from_numpy(player['defense_ranks'][:10]).view(1, 10, 1).float()
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['skill_score'][:10]).view(1, 10, 1).float()), dim=2)

    player_test_inputs = torch.from_numpy(player['defense_ranks'][-5:]).view(1, 5, 1).float()
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['skill_score'][-5:]).view(1, 5, 1).float()), dim=2)

    X_train.append(player_inputs)
    X_test.append(player_test_inputs)
    Y_train.append(torch.from_numpy(player['fantasy_scores'][:10]).view(1, 10, 1).float())
    Y_test.append(torch.from_numpy(player['fantasy_scores'][-5:]).view(1, 5, 1).float())

# Combine inputs and targets for all players
X_train = torch.cat(X_train, dim=0)
X_test = torch.cat(X_test, dim=0)
Y_train = torch.cat(Y_train, dim=0)
Y_test = torch.cat(Y_test, dim=0)

In [14]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Set hyperparameters
input_size = 2
hidden_size = 50
learning_rate = 0.001
num_epochs = 100

# Create the model
model = RegressionModel(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


In [23]:
# Training loop for each player with batch size of 2
batch_size = 2
num_players = len(X_train)
num_batches = num_players // batch_size

for batch in range(num_batches):
    batch_inputs = X_train[batch * batch_size: (batch + 1) * batch_size]
    batch_targets = Y_train[batch * batch_size: (batch + 1) * batch_size]

    # Training loop
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss for every epoch
        print(f"Batch {batch+1}/{num_batches}, Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


# Make predictions for the player's test inputs
with torch.no_grad():
    player_predicted = model(X_test)

print() # blank line
# Print the predictions and original scores for the player
for i in range(len(X_test[0])):
    print(f"Defense Rank: {X_test[0][i][0]}, Predicted Fantasy Score: {player_predicted[0][i].item()}, Actual Fantasy Score: {Y_test[0][i].item()}")
    print(f"Defense Rank: {X_test[1][i][0]}, Predicted Fantasy Score: {player_predicted[1][i].item()}, Actual Fantasy Score: {Y_test[1][i].item()}\n")


Batch 1/1, Epoch 1/100, Loss: 40.975120544433594
Batch 1/1, Epoch 2/100, Loss: 40.728172302246094
Batch 1/1, Epoch 3/100, Loss: 41.04917526245117
Batch 1/1, Epoch 4/100, Loss: 40.783687591552734
Batch 1/1, Epoch 5/100, Loss: 41.1120491027832
Batch 1/1, Epoch 6/100, Loss: 40.840389251708984
Batch 1/1, Epoch 7/100, Loss: 41.21495819091797
Batch 1/1, Epoch 8/100, Loss: 40.911739349365234
Batch 1/1, Epoch 9/100, Loss: 41.285865783691406
Batch 1/1, Epoch 10/100, Loss: 40.97325897216797
Batch 1/1, Epoch 11/100, Loss: 41.38776397705078
Batch 1/1, Epoch 12/100, Loss: 41.0642204284668
Batch 1/1, Epoch 13/100, Loss: 41.47453689575195
Batch 1/1, Epoch 14/100, Loss: 41.11719512939453
Batch 1/1, Epoch 15/100, Loss: 41.558712005615234
Batch 1/1, Epoch 16/100, Loss: 41.167762756347656
Batch 1/1, Epoch 17/100, Loss: 41.62176513671875
Batch 1/1, Epoch 18/100, Loss: 41.178810119628906
Batch 1/1, Epoch 19/100, Loss: 41.575538635253906
Batch 1/1, Epoch 20/100, Loss: 41.180335998535156
Batch 1/1, Epoch 21/